## <center><ins>Trabajo Práctico 1: Reservas de hotel</ins></center>
#### <center>*Checkpoint 3: Ensambles - Preprocesamiento*</center>
##### <center>75.06 / 95.58 Organización de Datos - FIUBA</center>
##### <center>Ing. Rodríguez - 1°C 2023</center>
---
#### <center>**Grupo 31 - "Datazo":**</center>
<center> Michael Mena - 102685</center>
<center>Pablo Brocca - 104256</center>
<center>Diego Kisinovsky - 106203</center>

---
---

## Dataset de entrenamiento

##### Importamos las librerias

In [143]:
import pandas as pd
import numpy as np
hotel = pd.read_csv('./dataset/hotels_train.csv')

##### Aplicamos el preprocesamiento de Checkpoint 1:

In [144]:
hotel.drop(hotel[hotel['adults'] == 0].index, inplace = True)
hotel.drop(hotel[hotel['adr'] == 0].index, inplace = True)
hotel.drop(hotel[hotel.days_in_waiting_list > 300].index, inplace=True)
hotel = hotel.drop('company',axis=1)
hotel['children'].fillna(0,inplace=True)
hotel['agent'] = hotel['agent'].fillna(0)
hotel = hotel.drop('reservation_status_date',axis=1)
hotel["stays_in_nights"] = hotel["stays_in_week_nights"] + hotel["stays_in_weekend_nights"]
hotel["total_of_guests"] = hotel["adults"] + hotel["children"] + hotel["babies"]
hotel['country'].fillna('PRT',inplace=True)

##### Terminamos de adaptar las variables más importantes para poder usar en el entrenamiento:

In [145]:

# Creamos la variable room_type_unchanged que indica si el tipo de habitación se mantuvo igual entre la reserva y la asignación
hotel["room_type_unchanged"] = hotel["reserved_room_type"] == hotel["assigned_room_type"]

# Convertimos las columnas con booleanos a 0 y 1
hotel["room_type_unchanged"] = hotel["room_type_unchanged"].astype(int)

# Cambiamos los meses por números
hotel["arrival_date_month"].replace(["January", "February", "March", "April", "May", "June",
                                     "July","August","September","October","November","December"],
                                     [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], inplace=True)

Como consideramos que es más pertinente al estudio del target el hecho de si se respetó el tipo de habitación hecho en la reserva, en lugar de los tipos de habitación en sí, para el entrenamiento dejamos de lado las variables _reserved_room_type_ y _assigned_room_type_ en favor de _room_type_unchanged_.

##### Target Encoding para la variable _country_:

In [146]:
# Obtener el recuento de cancelaciones y no cancelaciones por país
country_counts = hotel.groupby('country')['is_canceled'].value_counts().unstack().reset_index()

# Calcular la proporción de cancelaciones y no cancelaciones
country_counts['cancel_ratio'] = country_counts[1] / (country_counts[0] + country_counts[1])

# Calcular el peso ponderado
country_counts['weight'] = country_counts[0] / country_counts[1]

# Reemplazar valores infinitos con NaN
country_counts['weight'] = country_counts['weight'].replace([float('inf')], float('nan'))

# Calcular la media de 'country_encoded' sin considerar los valores nulos
mean_encoding = np.nanmean(country_counts['cancel_ratio'])

# Crear un diccionario de codificación ponderada y reemplazar los valores nulos con la media
encoding_dict = country_counts.set_index('country')['cancel_ratio'].fillna(mean_encoding).to_dict()

# Aplicar codificación ponderada a la columna 'country'
hotel['country_encoded'] = hotel['country'].map(encoding_dict)


##### One Hot Encoding para variables categóricas:

In [147]:
# Dejamos las k variables dummies de cada columna para evitar inconsistencias de orden con el dataframe de test
hotel_dummies = pd.get_dummies(
    hotel,
    columns=["customer_type", "deposit_type", "hotel",
             "distribution_channel", "market_segment", "meal"], drop_first=True) #Eliminamos las dommies sobrantes ya que no aportan información

dummies = hotel_dummies.drop(hotel_dummies[hotel.drop(["customer_type","deposit_type","hotel","distribution_channel",
                                                      "market_segment","meal"],axis=1).columns],axis=1)

dummies.columns.tolist()
# Todas estas columnas tienen valores binarios, falta incluir a "room_type_unchanged" y "country_encoded"
# Deberian ser ignoradas al momento de normalizar

['customer_type_Group',
 'customer_type_Transient',
 'customer_type_Transient-Party',
 'deposit_type_Non Refund',
 'deposit_type_Refundable',
 'hotel_Resort Hotel',
 'distribution_channel_Direct',
 'distribution_channel_GDS',
 'distribution_channel_TA/TO',
 'distribution_channel_Undefined',
 'market_segment_Complementary',
 'market_segment_Corporate',
 'market_segment_Direct',
 'market_segment_Groups',
 'market_segment_Offline TA/TO',
 'market_segment_Online TA',
 'market_segment_Undefined',
 'meal_FB',
 'meal_HB',
 'meal_SC',
 'meal_Undefined']

##### Preparación de features:

In [148]:
hotel_train = hotel_dummies.drop(['id', 'reserved_room_type', 'assigned_room_type',
                                  'country', 'arrival_date_week_number', 'adults', 'children',
                                  'babies', 'stays_in_week_nights', 'stays_in_weekend_nights'],axis=1)

In [149]:
hotel_train.loc[0]

lead_time                           49.000000
arrival_date_year                 2016.000000
arrival_date_month                   9.000000
arrival_date_day_of_month            5.000000
is_repeated_guest                    0.000000
previous_cancellations               0.000000
previous_bookings_not_canceled       0.000000
booking_changes                      0.000000
agent                                9.000000
days_in_waiting_list                 0.000000
adr                                115.500000
required_car_parking_spaces          0.000000
total_of_special_requests            1.000000
is_canceled                          1.000000
stays_in_nights                      3.000000
total_of_guests                      1.000000
room_type_unchanged                  1.000000
country_encoded                      0.339268
customer_type_Group                  0.000000
customer_type_Transient              1.000000
customer_type_Transient-Party        0.000000
deposit_type_Non Refund           

---

## Dataset de test

##### Lectura del dataset:

In [150]:
hotel_test_df = pd.read_csv('./dataset/hotels_test.csv')
hotel_test_df

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date,id
0,City Hotel,277,2016,November,46,7,1,2,2,0.0,...,Non Refund,NaN,NaN,0,Transient,100.00,0,0,2016-04-04,e6d0d58a-ab96-44d3-b28f-85df1cc15f1e
1,City Hotel,286,2016,May,20,11,0,4,2,0.0,...,No Deposit,30.0,NaN,176,Transient-Party,101.00,0,0,2016-01-22,aba9aac2-4495-4645-a60e-90328713c999
2,City Hotel,0,2016,August,35,23,0,1,3,0.0,...,No Deposit,NaN,NaN,0,Transient,225.00,0,2,2016-08-24,b345b5dc-c804-4d77-a39f-1f4e65d1f1ad
3,Resort Hotel,196,2016,October,44,29,2,5,2,2.0,...,No Deposit,240.0,NaN,0,Transient,96.00,0,0,2016-11-05,ce1b400e-afa1-4946-bdff-004b859eb61c
4,Resort Hotel,232,2016,November,48,26,2,1,2,0.0,...,Non Refund,11.0,NaN,0,Transient,30.00,0,0,2016-10-18,c4fbec0d-8e2e-4db2-9d4f-a6ccb9f51214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26530,City Hotel,31,2017,April,16,21,2,3,2,0.0,...,No Deposit,21.0,NaN,0,Transient-Party,85.00,0,0,2017-04-26,636ccded-66ad-4538-8006-c0b9f84c6256
26531,City Hotel,130,2016,September,36,3,2,1,2,0.0,...,No Deposit,42.0,NaN,0,Transient,80.75,0,0,2016-09-06,11e19741-bd28-4459-8ea3-184f1664000f
26532,Resort Hotel,45,2016,August,35,22,1,1,2,0.0,...,No Deposit,240.0,NaN,0,Transient,231.00,0,0,2016-08-04,36a4c7b0-d6b1-4528-93db-311be96ec1c6
26533,City Hotel,164,2017,August,32,10,2,4,2,0.0,...,No Deposit,85.0,NaN,0,Transient,80.10,0,0,2017-03-07,89c85fa5-2b3d-4131-844c-e151a65ca240


##### Copia del dataset para mantener id:

In [151]:
hotel_test = hotel_test_df.copy()

##### Se aplican los cambios del preprocesamiento:

In [152]:
hotel_test = hotel_test.drop('company',axis=1)
hotel_test['children'].fillna(0,inplace=True)
hotel_test['agent'] = hotel_test['agent'].fillna(0)
hotel_test['country'].fillna('PRT',inplace=True)
hotel_test = hotel_test.drop('reservation_status_date',axis=1)
hotel_test["stays_in_nights"] = hotel_test["stays_in_week_nights"] + hotel_test["stays_in_weekend_nights"]
hotel_test["total_of_guests"] = hotel_test["adults"] + hotel_test["children"] + hotel_test["babies"]
hotel_test["room_type_unchanged"] = hotel_test["reserved_room_type"] == hotel_test["assigned_room_type"]
hotel_test["arrival_date_month"].replace(["January", "February", "March", "April", "May", "June",
                                     "July","August","September","October","November","December"],
                                     [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], inplace=True)


In [153]:
# Convertimos las columnas con booleanos a 0 y 1
hotel_test["room_type_unchanged"] = hotel_test["room_type_unchanged"].astype(int)

##### Target Encoding para la variable _country_ del dataset de test:

In [154]:
# Como tenemos el target en el dataset de test no podemos aplicar la codificación ponderada
# Entonces utilizamos el diccionario de codificación ponderada del dataset de train
hotel_test['country_encoded'] = hotel_test['country'].map(encoding_dict)

In [155]:
# Aplicamos la codificación ponderada a los valores nulos
hotel_test['country_encoded'].fillna(mean_encoding,inplace=True)

##### One hot encoding para variables categóricas de test:

In [156]:
hotel_dummies_test = pd.get_dummies(hotel_test, columns=["customer_type", "deposit_type", "hotel",
                                                         "distribution_channel", "market_segment", "meal"], drop_first=True)

dummies_test = hotel_dummies_test.drop(hotel_dummies_test[hotel_test.drop(["customer_type","deposit_type","hotel","distribution_channel",
                                                                 "market_segment","meal"],axis=1).columns],axis=1)

dummies_test.columns.tolist()

['customer_type_Group',
 'customer_type_Transient',
 'customer_type_Transient-Party',
 'deposit_type_Non Refund',
 'deposit_type_Refundable',
 'hotel_Resort Hotel',
 'distribution_channel_Direct',
 'distribution_channel_GDS',
 'distribution_channel_TA/TO',
 'market_segment_Complementary',
 'market_segment_Corporate',
 'market_segment_Direct',
 'market_segment_Groups',
 'market_segment_Offline TA/TO',
 'market_segment_Online TA',
 'meal_FB',
 'meal_HB',
 'meal_SC',
 'meal_Undefined']

Quitamos las mismas variables que en el dataset de entrenamiento:

In [157]:
hotel_test = hotel_dummies_test.drop(['id', 'reserved_room_type', 'assigned_room_type','country',
                                      'arrival_date_week_number','adults', 'children','babies',
                                      'stays_in_week_nights', 'stays_in_weekend_nights'],axis=1)

Revisamos si quedaron variables que no están en el dataset de test pero sí en el de entrenamiento:

In [158]:
test_difference = [col for col in hotel_train.drop('is_canceled',axis=1).columns.tolist() if col not in hotel_test.columns.tolist()]
test_difference

['distribution_channel_Undefined', 'market_segment_Undefined']

Como son columnas dummies de valores que no aparecen, las agregamos al dataset de test con valor 0:

In [159]:
hotel_test['distribution_channel_Undefined'] = 0
hotel_test['market_segment_Undefined'] = 0

hotel_test[['distribution_channel_Undefined','market_segment_Undefined']].head(10)

,distribution_channel_Undefined,market_segment_Undefined
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


---

## Exportación de datasets preprocesados

In [160]:
# Entrenamiento
hotel_train.to_csv('./dataset/hotels_train_preprocesado.csv', index=False)

# Test
hotel_test.to_csv('./dataset/hotels_test_preprocesado.csv', index=False)